In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%load_ext autotime

# Sample Submission

In [ ]:
#sample = pd.read_csv("sample_submission.csv")
#sample[:3]

In [ ]:
#sample.prediction.value_counts()

# Test

In [ ]:
#test_orig = pd.read_csv("test.csv")
#test_orig[:3]

# Train Cleaning

In [2]:
train_orig = pd.read_csv("/home/sroberts/train.csv", engine='python')

time: 58.3 s


In [ ]:
#y_train = train_orig['target'].copy()
#x_train = train_orig.loc[:, train_orig.columns != 'target'].copy()
#x_train = train_orig.loc[:, ~train_orig.columns.isin(['target','severe_toxicity','obscene',
#                                                    'identity_attack','insult','threat'])].copy()

In [3]:
train_orig[:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.0,Thank you!! This would make my life a lot less...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4


time: 38.9 ms


In [4]:
# toxic comments
train_orig[train_orig.target >= 0.5][:2]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:50.865549+00,2,NaN,2006,rejected,0,0,0,0,0,0.009524,0,105


time: 118 ms


In [ ]:
#x_train[:2]

# Fill NaNs with Zeros

In [ ]:
#x_train.fillna(0, inplace=True)

In [5]:
train_orig.fillna(0, inplace=True)

time: 812 ms


# describe/summarize

In [ ]:
desc_df = train_orig.copy()
desc_df.loc[desc_df.target >= 0.5, "toxic"] = True
desc_df.toxic.fillna(False,inplace=True)

In [ ]:
rel_col = list(set(desc_df.columns.tolist()) - set(['target','id']))
desc_df[rel_col].groupby("toxic").describe()

# lower case, stopwords, puncutation

In [ ]:
#import nltk
#nltk.download('all')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')

In [6]:
pd.options.display.max_colwidth = 1000
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
import string

time: 497 ms


In [7]:
#import nltk
#nltk.download('stopwords')

english_stopwords = set(stopwords.words('english'))
w_tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

time: 2.34 ms


In [8]:
def clean_message(msg):
    msg = msg.lower()
    msg_tokens = nltk.word_tokenize(msg)
    clean_msg_tokens = [w for w in msg_tokens if w not in english_stopwords]
    clean_msg_tokens_puct = [w for w in clean_msg_tokens if w not in string.punctuation]
    lemmatized_token = [lemmatizer.lemmatize(w) for w in clean_msg_tokens_puct]
    return lemmatized_token

time: 7.96 ms


In [ ]:
# poorn
#train_orig['clean'] = train_orig.comment_text.apply(clean_message)

In [ ]:
#nltk.download('wordnet')


In [9]:
train_orig["clean"] = train_orig.comment_text.apply(lambda x: " ".join([words.strip(string.punctuation).lower()
for words in [word for word in x.split() if (word not in string.punctuation) & (word not in english_stopwords)]]))
train_orig["clean"] = train_orig["clean"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) 
                                                                    for w in w_tokenizer.tokenize(x)]))

time: 4min 34s


In [10]:
train_orig['clean'][:2]

0          this cool it's like would want mother read this really great idea well done
1    thank you this would make life lot le anxiety-inducing keep up let anyone get way
Name: clean, dtype: object

time: 4.48 ms


# vectorize

In [11]:
from collections import Counter
import numpy as np

time: 7.9 ms


In [ ]:
# pooran
#voc_counter = Counter(messages_df.clean_message.sum())

In [12]:
voc_counter = Counter(" ".join(train_orig.clean).split(" "))

time: 16.5 s


In [13]:
#for w,c in voc_counter.items():
#    print(w,c)

time: 262 µs


In [14]:
voc_set = dict((k, v) for k, v in voc_counter.items() if v > 10)
voc_dict = {w:i for w,i in zip(voc_set, range(len(voc_set)))}

time: 160 ms


In [15]:
train_orig["clean"] = train_orig["clean"].apply(lambda x: x.split())
train_orig.loc[train_orig.target >= 0.5, "toxic"] = True
train_orig.toxic.fillna(False,inplace=True)

time: 12.3 s


In [ ]:
#train_orig = train_orig.sample(500000)

In [16]:
X = np.zeros((len(train_orig), len(voc_set)), dtype = np.int8 )
for msg_idx, msg in enumerate(train_orig['clean']):
    for word in set(msg):
        if word in voc_dict:
            X[msg_idx][voc_dict[word]] = msg.count(word)
            
y = np.where(train_orig.toxic == True, 1, 0)

time: 1min 33s


In [ ]:
#from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler(random_state=0)
#X_resampled, y_resampled = ros.fit_resample(X, y)

# Save

In [ ]:
X.shape

In [ ]:
# np.savetxt('y.txt', y, fmt='%d')
# np.savetxt('X.txt', X, fmt='%d')

# !!! restart kernel !!!

# Load

In [ ]:
# import numpy as np
# X = np.loadtxt('X.txt', dtype=int)
# y = np.loadtxt('y.txt', dtype=int)

# Model

# SGDClassifier

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

time: 559 µs


In [18]:
from sklearn.linear_model import SGDClassifier

time: 7.97 ms


In [ ]:
#train_size=.8
#train_indicies = int(X.shape[0]*train_size)

#np.random.shuffle(X)
#train_X,test_X = X[:train_indicies,:],X[train_indicies:,:]
#train_y,test_y = y[:train_indicies],  y[train_indicies:]

In [19]:
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=.2,random_state=2)

time: 1min 27s


In [20]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(train_X, train_y)

/home/sroberts/anaconda3/envs/toxic/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


time: 1min 45s


In [21]:
lr = SGDClassifier(loss="log", n_jobs=-1, class_weight={0:})

#classes = np.unique(["toxic", "not_toxic"])
classes = np.unique([0, 1])

time: 860 µs


In [ ]:
# minibatches = [(train_X[:100000,:],train_y[:100000]),
#                (train_X[100001:200000,:],train_y[100001:200000]),
#                (train_X[200001:300000,:],train_y[200001:300000]),
#                (train_X[300001:400000,:],train_y[300001:400000]),
#                (train_X[400001:500000,:],train_y[400001:500000]),
#                (train_X[500001:600000,:],train_y[500001:600000]),
#                (train_X[600001:700000,:],train_y[600001:700000]),
#                (train_X[700001:800000,:],train_y[700001:800000]),
#                (train_X[800001:900000,:],train_y[800001:900000]),
#                (train_X[900001:1000000,:],train_y[900001:1000000]),
#                (train_X[1000001:1100000,:],train_y[1000001:1100000]),
#                (train_X[1100001:1200000,:],train_y[1100001:1200000]),
#                (train_X[1200001:1300000,:],train_y[1200001:1300000]),
#                (train_X[1300001:1400000,:],train_y[1300001:1400000]),
#                (train_X[1400001:,:],train_y[1400001:])]

In [22]:
minibatches = [(X_resampled[:100000,:],y_resampled[:100000]),
               (X_resampled[100001:200000,:],y_resampled[100001:200000]),
               (X_resampled[200001:300000,:],y_resampled[200001:300000]),
               (X_resampled[300001:400000,:],y_resampled[300001:400000]),
               (X_resampled[400001:500000,:],y_resampled[400001:500000]),
               (X_resampled[500001:600000,:],y_resampled[500001:600000]),
               (X_resampled[600001:700000,:],y_resampled[600001:700000]),
               (X_resampled[700001:800000,:],y_resampled[700001:800000]),
               (X_resampled[800001:900000,:],y_resampled[800001:900000]),
               (X_resampled[900001:1000000,:],y_resampled[900001:1000000]),
               (X_resampled[1000001:1100000,:],y_resampled[1000001:1100000]),
               (X_resampled[1100001:1200000,:],y_resampled[1100001:1200000]),
               (X_resampled[1200001:1300000,:],y_resampled[1200001:1300000]),
               (X_resampled[1300001:1400000,:],y_resampled[1300001:1400000]),
               (X_resampled[1400001:,:],y_resampled[1400001:])]

time: 5.32 ms


In [23]:
for xs, ys in minibatches:
    lr.partial_fit(xs, ys, classes=classes)

MemoryError: 

time: 14min 31s


In [ ]:
lr.score(test_X, test_y)

In [ ]:
lr.coef_ef_

In [ ]:
lr.intercept_

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
lr.loss_function_

In [ ]:
lr.n_iter_

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# Save Model

In [ ]:
import pickle
# save the classifier
with open('my_dumped_classifier.pkl', 'wb') as fid:
    pickle.dump(lr, fid)    




In [ ]:
# load it again
with open('my_dumped_classifier.pkl', 'rb') as fid:
    lr_loaded = pickle.load(fid)

In [ ]:
lr_loaded.score(test_X, test_y)

Cool, still works

# Test data

In [ ]:
test_orig = pd.read_csv("/home/sroberts/test.csv", engine='python')

In [ ]:
test_orig[:2]

In [ ]:
test_orig.fillna(0, inplace=True)
test_orig['clean'] = test_orig.comment_text.apply(lambda x: " ".join([words.strip(string.punctuation).lower()
for words in [word for word in x.split() if (word not in string.punctuation) & (word not in english_stopwords)]]))
test_orig["clean"] = test_orig["clean"].apply(lambda x: " ".join([lemmatizer.lemmatize(w) 
                                                                    for w in w_tokenizer.tokenize(x)]))
#voc_counter = Counter(" ".join(test_orig.clean).split(" "))
#voc_set = dict((k, v) for k, v in voc_counter.items() if v > 10)
#voc_dict = {w:i for w,i in zip(voc_set, range(len(voc_set)))}
test_orig["clean"] = test_orig["clean"].apply(lambda x: x.split())

In [ ]:
#test_orig.loc[test_orig.target >= 0.5, "toxic"] = True
#test_orig.toxic.fillna(False,inplace=True)
X = np.zeros((len(test_orig), len(voc_set)), dtype = np.int8 )
for msg_idx, msg in enumerate(test_orig['clean']):
    for word in set(msg):
        if word in voc_dict:
            X[msg_idx][voc_dict[word]] = msg.count(word)
            
#y = np.where(test_orig.toxic == True, 1, 0)

In [ ]:
X.shape

# MLP Classifier

In [24]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

time: 4.79 ms


In [ ]:
#train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=.2,random_state=2)

In [25]:
clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
#classes = np.unique(["toxic", "not_toxic"])
classes = np.unique([0, 1])

time: 862 µs


In [ ]:
# minibatches = [(train_X[:100000,:],train_y[:100000]),
#                (train_X[100001:200000,:],train_y[100001:200000]),
#                (train_X[200001:300000,:],train_y[200001:300000]),
#                (train_X[300001:400000,:],train_y[300001:400000]),
#                (train_X[400001:500000,:],train_y[400001:500000]),
#                (train_X[500001:600000,:],train_y[500001:600000]),
#                (train_X[600001:700000,:],train_y[600001:700000]),
#                (train_X[700001:800000,:],train_y[700001:800000]),
#                (train_X[800001:900000,:],train_y[800001:900000]),
#                (train_X[900001:1000000,:],train_y[900001:1000000]),
#                (train_X[1000001:1100000,:],train_y[1000001:1100000]),
#                (train_X[1100001:1200000,:],train_y[1100001:1200000]),
#                (train_X[1200001:1300000,:],train_y[1200001:1300000]),
#                (train_X[1300001:1400000,:],train_y[1300001:1400000]),
#                (train_X[1400001:,:],train_y[1400001:])]

In [26]:
for xs, ys in minibatches:
    clf.partial_fit(xs, ys, classes=classes)

time: 58min 44s


In [27]:
clf.score(test_X, test_y)

0.07974236442966964

time: 2min 27s


In [28]:
clf.coefs_

[array([[ 0.01401458,  0.00463394,  0.00501281,  0.01094861, -0.00292222],
        [-0.0082903 , -0.00609479, -0.00306301, -0.00233737,  0.00090535],
        [ 0.00309259,  0.00380837, -0.00150815,  0.00792303, -0.00715011],
        ...,
        [ 0.00489398, -0.00787083, -0.00903126, -0.00810709,  0.00217807],
        [-0.00248438, -0.00478913,  0.00449138, -0.00571414, -0.00093135],
        [-0.00548875, -0.00639404,  0.00365674,  0.00561295,  0.00538747]]),
 array([[-0.858365  ,  0.24408481],
        [ 0.01992537, -0.8862989 ],
        [-0.55352744, -0.34125225],
        [-0.70994201, -0.40900862],
        [ 0.11107635, -0.84903885]]),
 array([[-1.00336856],
        [ 0.72206879]])]

time: 3.14 ms


In [29]:
clf.intercepts_

[array([0.12492733, 0.01214916, 0.08904607, 0.10782317, 0.03537445]),
 array([ 0.3985795 , -0.06732792]),
 array([3.27367642])]

time: 9.33 ms


save model

In [33]:
import pickle
with open('mlpclassifier1_imblearn.pkl', 'wb') as fid:
    pickle.dump(clf, fid)    


time: 11.3 ms


In [ ]:
with open('mlpclassifier1_imblearn.pkl', 'rb') as fid:
    mlp_loaded = pickle.load(fid)

# Naive Bayes

In [30]:
from sklearn.naive_bayes import MultinomialNB

time: 9.46 ms


In [31]:
nb = MultinomialNB()


time: 5.47 ms


In [34]:
for xs, ys in minibatches:
    nb.partial_fit(xs, ys, classes=classes)

MemoryError: 

time: 15min 38s


In [ ]:
nb.score(test_X, test_y)

In [ ]:
nb.intercept_

In [ ]:
nb.coef_

In [ ]:
nb.feature_count_

In [ ]:
nb.class_count_

save model

In [ ]:
with open('naivebayes_weights.pkl', 'wb') as fid:
    pickle.dump(nb, fid) 

In [ ]:
with open('naivebayes_weights.pkl', 'rb') as fid:
    nb_loaded = pickle.load(fid)

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(warm_start=True, n_estimators=1, n_jobs=-1)

In [ ]:
for xs, ys in minibatches:
    rf.fit(xs,ys)
    rf.n_estimators += 1
    #nb.partial_fit(xs, ys, classes=classes)

In [ ]:
rf.score(test_X, test_y)

In [ ]:
rf.estimators_

In [ ]:
rf.n_features_

In [ ]:
rf.n_outputs_

In [ ]:
rf.feature_importances_

save model

In [ ]:
import pickle
with open('randomforest.pkl', 'wb') as fid:
    pickle.dump(rf, fid) 

In [ ]:
with open('randomforest.pkl', 'rb') as fid:
    rf_loaded = pickle.load(fid)

# Play with RF parameters

In [ ]:
rf2 = RandomForestClassifier(warm_start=True, n_estimators=200, max_depth=8, n_jobs=-1)

In [ ]:
for xs, ys in minibatches:
    rf2.fit(xs,ys)
    rf2.n_estimators += 1

In [ ]:
rf2.score(test_X, test_y)

In [ ]:
print(rf2.estimators_)
print(rf2.n_features_)
print(rf2.n_outputs_)
print(rf2.feature_importances_)

In [ ]:
import pickle
with open('randomforest2.pkl', 'wb') as fid:
    pickle.dump(rf2, fid) 

In [ ]:
with open('randomforest2.pkl', 'rb') as fid:
    rf2_loaded = pickle.load(fid)

# Test sets after passing through models

In [ ]:
sgdc_test = lr_loaded.predict(X)

In [ ]:
mlp_test = mlp_loaded.predict(X)

In [ ]:
nb_test = nb_loaded.predict(X)

In [ ]:
rf_test = rf_loaded.predict(X)

In [ ]:
rf2_test = rf2_loaded.predict(X)

In [ ]:
with open('sgdc_test.pkl', 'wb') as fid:
    pickle.dump(sgdc_test, fid) 

In [ ]:
with open('mlp_test.pkl', 'wb') as fid:
    pickle.dump(mlp_test, fid) 

In [ ]:
with open('nb_test.pkl', 'wb') as fid:
    pickle.dump(nb_test, fid) 

In [ ]:
with open('rf_test.pkl', 'wb') as fid:
    pickle.dump(rf_test, fid) 

In [ ]:
with open('rf2_test.pkl', 'wb') as fid:
    pickle.dump(rf2_test, fid) 

In [ ]:
train_orig.loc[train_orig.target >= 0.5, "toxic"] = True

In [ ]:
train_orig.toxic.fillna(False,inplace=True)

In [ ]:
train_orig.toxic.value_counts()